In [1]:
import pandas as pd
import os
from pathlib import Path
import re
import spacy
import numpy as np
from collections import defaultdict
pd.set_option('max.columns',100)

In [2]:
raw = Path(os.getcwd()) / '..' / '..'/ 'data' / '01_raw'

### Join kof with sc (sneaker)

In [3]:
kof = pd.read_csv(raw / 'kof.csv')
sneaker = pd.read_csv(raw / 'sneaker.csv')

kof['name_join'] = kof['name'].str.lower()
kof['name_join'] = kof['name_join'].str.replace('[^\w\s]','')

sneaker['name_join'] = sneaker['name'].str.lower()
sneaker['name_join'] = sneaker['name_join'].str.replace('[^\w\s]','')

df = kof.merge(sneaker[['style_code', 'description', 'original_price', 'brand_id', 
         'secondary_brand_id', 'silhouette_id']], 
               how='left', on='style_code')

df = df.merge(sneaker[['name_join', 'description', 'original_price', 'brand_id', 
         'secondary_brand_id', 'silhouette_id', 'style_code']], 
               how='left', on='name_join')

kof['style_code'] = kof['style_code'].replace('\n', np.nan)
kof['style_code'] = kof['style_code'].replace(np.nan, 'UNKNOWN')

df['style_code_x'] = df['style_code_x'].fillna(df['style_code_y'])
df['description_x'] = df['description_x'].fillna(df['description_y'])
df['original_price_x'] = df['original_price_x'].fillna(df['original_price_y'])
df['brand_id_x'] = df['brand_id_x'].fillna(df['brand_id_y'])
df['secondary_brand_id_x'] = df['secondary_brand_id_x'].fillna(df['secondary_brand_id_y'])
df['silhouette_id_x'] = df['silhouette_id_x'].fillna(df['silhouette_id_y'])

df = df.drop(columns=['brand_id_y', 'secondary_brand_id_y', 'silhouette_id_y', 
                      'description_y', 'original_price_y', 'style_code_y'])
df = df.rename(columns={'brand_id_x': 'brand_id', 'secondary_brand_id_x':'secondary_brand_id',
                       'silhouette_id_x':'silhouette_id', 'description_x':'description',
                       'original_price_x':'original_price', 'style_code_x':'style_code'})

In [4]:
df.head(1)

,style_code,name,brand,date,retail_price,colorway,story,wants,name_join,description,original_price,brand_id,secondary_brand_id,silhouette_id
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220.0,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,nike air foamposite one snakeskin,"The Nike Air Foamposite One ""Snakeskin"" will b...",230,117.0,397.0,685.0


### Join Brand

In [5]:
brand = pd.read_csv(raw / 'brand.csv')

brand['name'] = brand['name'].str.title()

brand = brand[['id', 'name', 'lines_count', 'models_count', 'releases_count', 'year_founded']]

brand = brand.append({'id':9, 'name':'Air Jordan', 'lines_count':11, 'models_count':139, 
              'releases_count':1653, 'year_founded':1984}, ignore_index=True)
brand = brand.append({'id':10, 'name':'Jordan Brand', 'lines_count':11, 'models_count':139, 
              'releases_count':1653, 'year_founded':1984}, ignore_index=True)

brand = brand.rename(columns={'id':'brand_id', 'name':'brand'})

df = df.merge(brand[['brand_id', 'brand']], how='left', on='brand')
df['brand_id_x'] = df['brand_id_x'].fillna(df['brand_id_y'])

df = df.drop(columns='brand_id_y')
df = df.rename(columns={'brand_id_x': 'brand_id'})

brand = brand.drop(columns='brand')

df = df.merge(brand, how='left', on='brand_id')

df = df.rename(columns={'lines_count':'brand_lines_count', 'models_count':'brand_models_count', 
                   'releases_count':'brand_releases_count', 'year_founded':'brand_year_founded'})

In [6]:
df.head(1)

,style_code,name,brand,date,retail_price,colorway,story,wants,name_join,description,original_price,brand_id,secondary_brand_id,silhouette_id,brand_lines_count,brand_models_count,brand_releases_count,brand_year_founded
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220.0,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,nike air foamposite one snakeskin,"The Nike Air Foamposite One ""Snakeskin"" will b...",230,117.0,397.0,685.0,26.0,536.0,6024.0,1964.0


### Join Secondary Brand

In [7]:
secondary_brand = pd.read_csv(raw / 'secondary_brand.csv')

secondary_brand = secondary_brand[['id', 'name', 'models_count', 'releases_count']]

secondary_brand = secondary_brand.rename(columns={'id':'secondary_brand_id', 'name':'secondary_brand', 
                                'models_count':'sb_models_count', 'releases_count':'sb_releases_count',
                                                 'lines_count':'sb_lines_count'})

df = df.merge(secondary_brand, how='left', on='secondary_brand_id')

In [8]:
df.head(1)

,style_code,name,brand,date,retail_price,colorway,story,wants,name_join,description,original_price,brand_id,secondary_brand_id,silhouette_id,brand_lines_count,brand_models_count,brand_releases_count,brand_year_founded,secondary_brand,sb_models_count,sb_releases_count
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220.0,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,nike air foamposite one snakeskin,"The Nike Air Foamposite One ""Snakeskin"" will b...",230,117.0,397.0,685.0,26.0,536.0,6024.0,1964.0,Nike Basketball,68.0,444.0


### Join Silhouette

In [9]:
silhouette = pd.read_csv(raw / 'silhouette.csv')

silhouette = silhouette[['id','name','releases_count','models_count']]
silhouette = silhouette.rename(columns={'id':'silhouette_id', 'name':'silhouette', 
                           'releases_count':'s_releases_count', 'models_count':'s_models_count'})

df = df.merge(silhouette, how='left', on='silhouette_id')

In [10]:
df.head(1)

,style_code,name,brand,date,retail_price,colorway,story,wants,name_join,description,original_price,brand_id,secondary_brand_id,silhouette_id,brand_lines_count,brand_models_count,brand_releases_count,brand_year_founded,secondary_brand,sb_models_count,sb_releases_count,silhouette,s_releases_count,s_models_count
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220.0,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,nike air foamposite one snakeskin,"The Nike Air Foamposite One ""Snakeskin"" will b...",230,117.0,397.0,685.0,26.0,536.0,6024.0,1964.0,Nike Basketball,68.0,444.0,Nike Air Foamposite One,65.0,NaN


### Feature Engineering

In [11]:
df['date'] = pd.to_datetime(df['date'])

In [12]:
df['release_month'] = df['date'].dt.month
df['release_dow'] = df['date'].dt.weekday
month = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
dow = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
df['release_month'] = df['release_month'].map(month)
df['release_dow'] = df['release_dow'].map(dow)

In [13]:
df['wmns'] = df['name_join'].str.contains(r' wmns')
df['collab'] = df['name_join'].str.contains(r' x ')
df['retro'] = df['name_join'].str.contains(r' retro| og')
df['kids'] = df['name_join'].str.contains(r' gs| kid| kids')

In [14]:
df['colorway_nlp'] = df['colorway'].str.lower()
df['colorway_nlp'] = df['colorway_nlp'].str.replace('[^\w\s]',' ')

In [15]:
nlp = spacy.load("en_core_web_lg")
black_token = nlp("black")
white_token = nlp("white")
brown_token = nlp("brown")
gray_token = nlp("gray")
red_token = nlp("red")
orange_token = nlp("orange")
yellow_token = nlp("yellow")
green_token = nlp("green")
blue_token = nlp("blue")
purple_token = nlp("purple")
mc_token = nlp("multi color")

color_dict = defaultdict(list)

for colorway in list(df.colorway_nlp):
    colorway = colorway.lower()
    colorway = re.sub('[^\w\s]', ' ', colorway)
    color_token = nlp(colorway)

    color_dict['Black'].append(color_token.similarity(black_token))
    color_dict['White'].append(color_token.similarity(white_token))
    color_dict['Brown'].append(color_token.similarity(brown_token))
    color_dict['Gray'].append(color_token.similarity(gray_token))
    color_dict['Red'].append(color_token.similarity(red_token))
    color_dict['Orange'].append(color_token.similarity(orange_token))
    color_dict['Yellow'].append(color_token.similarity(yellow_token))
    color_dict['Green'].append(color_token.similarity(green_token))
    color_dict['Blue'].append(color_token.similarity(blue_token))
    color_dict['Purple'].append(color_token.similarity(purple_token))
    color_dict['Multi-Color'].append(color_token.similarity(mc_token))

/Users/nikhilbhargava/opt/anaconda3/envs/mids-python/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
/Users/nikhilbhargava/opt/anaconda3/envs/mids-python/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
/Users/nikhilbhargava/opt/anaconda3/envs/mids-python/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
/Users/nikhilbhargava/opt/anaconda3/envs/mids-python/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
/Users/nikhilbhargava/opt/anaconda3/envs/mids-python/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
/Users/nikhilbhargava/opt/anaconda3/envs/mids-python/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarn

In [16]:
colors_df = pd.DataFrame(color_dict)

In [17]:
colors_df.columns

Index(['Black', 'White', 'Brown', 'Gray', 'Red', 'Orange', 'Yellow', 'Green',
       'Blue', 'Purple', 'Multi-Color'],
      dtype='object')

In [18]:
colors_df['main_color'] = colors_df[['Black', 'White', 'Brown', 'Gray', 'Red', 'Orange', 'Yellow', 'Green',
       'Blue', 'Purple', 'Multi-Color']].idxmax(axis=1)

In [19]:
df['main_color'] = colors_df.main_color

In [20]:
df.head()

,style_code,name,brand,date,retail_price,colorway,story,wants,name_join,description,original_price,brand_id,secondary_brand_id,silhouette_id,brand_lines_count,brand_models_count,brand_releases_count,brand_year_founded,secondary_brand,sb_models_count,sb_releases_count,silhouette,s_releases_count,s_models_count,release_month,release_dow,wmns,collab,retro,kids,colorway_nlp,main_color
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220.0,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,nike air foamposite one snakeskin,"The Nike Air Foamposite One ""Snakeskin"" will b...",230,117.0,397.0,685.0,26.0,536.0,6024.0,1964.0,Nike Basketball,68.0,444.0,Nike Air Foamposite One,65.0,NaN,Apr,Monday,False,False,False,False,sail black habanero red black,Black
1,BV4580-400,Undercover x Nike SFB Mountain Obsidian Univer...,Nike,2019-04-15,350.0,Obsidian/University Red-Dark Obsidian,The Undercover x Nike SFB Mountain Obsidian/Un...,37,undercover x nike sfb mountain obsidian univer...,NaN,NaN,117.0,NaN,NaN,26.0,536.0,6024.0,1964.0,NaN,NaN,NaN,NaN,NaN,NaN,Apr,Monday,False,True,False,False,obsidian university red dark obsidian,Blue
2,BV4580-001,Undercover x Nike SFB Mountain Black,Nike,2019-04-15,350.0,Black/Sail-Black,The Undercover x Nike SFB Mountain Black is In...,70,undercover x nike sfb mountain black,NaN,NaN,117.0,NaN,NaN,26.0,536.0,6024.0,1964.0,NaN,NaN,NaN,NaN,NaN,NaN,Apr,Monday,False,True,False,False,black sail black,Black
3,AO2607-900,Nike PG 3 Mamba Mentality,Nike,2019-04-13,110.0,Multi-Color/Opti Yellow,The Nike PG 3 Mamba Mentality comes covered in...,179,nike pg 3 mamba mentality,"In celebration of Kobe Bryant's Mamba Day, the...",110,117.0,8295.0,10245.0,26.0,536.0,6024.0,1964.0,Nike Paul George,5.0,52.0,Nike PG 3,12.0,NaN,Apr,Saturday,False,False,False,False,multi color opti yellow,Multi-Color
4,AO2918-102,Nike Kyrie 5 Mamba Mentality,Nike,2019-04-13,130.0,White/Cyber,The Nike Kyrie 5 Mamba Mentality is a new rend...,271,nike kyrie 5 mamba mentality,"Inspired by Kobe Bryant's classic ""Chaos"" Nike...",130,117.0,397.0,10092.0,26.0,536.0,6024.0,1964.0,Nike Basketball,68.0,444.0,Nike Kyrie 5,28.0,NaN,Apr,Saturday,False,False,False,False,white cyber,White


### Join stockx

In [34]:
stockx = pd.read_csv(raw / 'stockx.csv')

In [36]:
stockx['resale_price'] = stockx['resale_price'].str.replace('[^\w\s]','')
stockx['resale_price'] = stockx['resale_price'].astype('float')

In [38]:
df = df.merge(stockx, how='left', on='style_code')

### Save as train.csv

In [39]:
intermed = Path(os.getcwd()) / '..' / '..'/ 'data' / '02_intermediate'
df.to_csv(intermed / 'train.csv', index=False)